In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['price'] = boston.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
def get_update_weights_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    N = target_batch.shape[0]
    pred_batch = w1*rm_batch + w2*lstat_batch + bias
    diff_batch = target_batch - pred_batch
    bias_factors = np.ones((N,))
    
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    return bias_update, w1_update, w2_update

### batch 경사 하강법
- 처음 batch 경사 하강법에 대한 이론이 나왔을 때의 형태 (지금은 이런식으로 많이 안씀)
- iter_epoch마다 임의의 batch_size 만큼 데이터를 뽑아서 weight를 update시켜줌

In [4]:
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    print('최초 w1, w2, bias: ', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:,0]
    lstat = features[:,1]
    
    for i in range(iter_epochs):
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        bias_update, w1_update, w2_update = get_update_weights_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch : ',i+1,'/', iter_epochs)
            pred = w1*rm + w2*lstat + bias
            diff = target - pred
            mse_loss = np.mean(np.square(diff))
            print('w1:',w1,'w2:',w2,'bias:',bias,'loss:',mse_loss)
    return w1, w2, bias

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['RM','LSTAT']])

w1, w2, bias = batch_gradient_descent(scaled_features, df['price'].values, iter_epochs=1000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #####')
print(w1, w2, bias)

최초 w1, w2, bias:  [0.] [0.] [0.]
Epoch :  1 / 1000
w1: [0.25719494] w2: [0.10040717] bias: [0.45846667] loss: 564.2879298354896
Epoch :  2 / 1000
w1: [0.45912978] w2: [0.20519104] bias: [0.84751868] loss: 541.4488608654348
Epoch :  3 / 1000
w1: [0.75472473] w2: [0.29177321] bias: [1.33577128] loss: 513.0280668599471
Epoch :  4 / 1000
w1: [0.95547681] w2: [0.38781562] bias: [1.71752173] loss: 491.8529646080828
Epoch :  5 / 1000
w1: [1.19300146] w2: [0.48839937] bias: [2.13332833] loss: 469.01294282055574
Epoch :  6 / 1000
w1: [1.47247799] w2: [0.58061269] bias: [2.60552074] loss: 443.8491355687481
Epoch :  7 / 1000
w1: [1.67091194] w2: [0.66526921] bias: [2.96943679] loss: 425.26643624889863
Epoch :  8 / 1000
w1: [1.87197828] w2: [0.7416014] bias: [3.32554632] loss: 407.4660500504585
Epoch :  9 / 1000
w1: [2.04365928] w2: [0.82080024] bias: [3.63470593] loss: 392.34822748747933
Epoch :  10 / 1000
w1: [2.28432497] w2: [0.90116753] bias: [4.02365481] loss: 373.50357408886083
Epoch :  11 /

w1: [10.95944994] w2: [0.79057201] bias: [16.31875902] loss: 69.67567885540994
Epoch :  161 / 1000
w1: [10.93253918] w2: [0.72915186] bias: [16.24497207] loss: 69.63210806703692
Epoch :  162 / 1000
w1: [10.93826379] w2: [0.71726194] bias: [16.24875855] loss: 69.59392828728193
Epoch :  163 / 1000
w1: [10.9579444] w2: [0.70010818] bias: [16.25595997] loss: 69.51814187083988
Epoch :  164 / 1000
w1: [10.97088119] w2: [0.66738608] bias: [16.24458682] loss: 69.43363230270795
Epoch :  165 / 1000
w1: [11.029708] w2: [0.65142392] bias: [16.30773004] loss: 69.26050350583873
Epoch :  166 / 1000
w1: [11.04761964] w2: [0.62654729] bias: [16.32581035] loss: 69.16734297137334
Epoch :  167 / 1000
w1: [11.07615676] w2: [0.60349544] bias: [16.34773148] loss: 69.06062272343449
Epoch :  168 / 1000
w1: [11.09658376] w2: [0.58485115] bias: [16.35849899] loss: 68.98262822924909
Epoch :  169 / 1000
w1: [11.08262253] w2: [0.55564979] bias: [16.32116108] loss: 68.9575713546851
Epoch :  170 / 1000
w1: [11.105030

w1: [13.10518746] w2: [-2.68515256] bias: [16.62242244] loss: 59.276507955601794
Epoch :  322 / 1000
w1: [13.07978577] w2: [-2.71733375] bias: [16.57103271] loss: 59.228220831182185
Epoch :  323 / 1000
w1: [13.10189937] w2: [-2.72865537] bias: [16.58604709] loss: 59.1822673726337
Epoch :  324 / 1000
w1: [13.09453809] w2: [-2.76010975] bias: [16.55297419] loss: 59.12273806864089
Epoch :  325 / 1000
w1: [13.11319671] w2: [-2.78603023] bias: [16.55480342] loss: 59.04882129683868
Epoch :  326 / 1000
w1: [13.10416862] w2: [-2.82173395] bias: [16.52158269] loss: 58.986805140968606
Epoch :  327 / 1000
w1: [13.1387125] w2: [-2.83049306] bias: [16.56357274] loss: 58.92993743608129
Epoch :  328 / 1000
w1: [13.15621285] w2: [-2.85217876] bias: [16.5669596] loss: 58.8664241534957
Epoch :  329 / 1000
w1: [13.16770884] w2: [-2.8635606] bias: [16.57846395] loss: 58.83156032118999
Epoch :  330 / 1000
w1: [13.1798929] w2: [-2.88402806] bias: [16.57800286] loss: 58.77624325609495
Epoch :  331 / 1000
w1:

w1: [14.85715743] w2: [-5.85603816] bias: [16.43264242] loss: 51.42234272599618
Epoch :  489 / 1000
w1: [14.88625128] w2: [-5.8601835] bias: [16.46649051] loss: 51.37796517363792
Epoch :  490 / 1000
w1: [14.8791877] w2: [-5.87240482] bias: [16.44640501] loss: 51.36807655996771
Epoch :  491 / 1000
w1: [14.8767389] w2: [-5.8899898] bias: [16.42519154] loss: 51.345946847413096
Epoch :  492 / 1000
w1: [14.87359434] w2: [-5.91305714] bias: [16.39548257] loss: 51.31980391334818
Epoch :  493 / 1000
w1: [14.88960531] w2: [-5.92469048] bias: [16.41049268] loss: 51.27804747474114
Epoch :  494 / 1000
w1: [14.88431787] w2: [-5.94038142] bias: [16.38820603] loss: 51.26528716786877
Epoch :  495 / 1000
w1: [14.88646587] w2: [-5.95719335] bias: [16.39272863] loss: 51.23399863957826
Epoch :  496 / 1000
w1: [14.90281136] w2: [-5.97582191] bias: [16.39125729] loss: 51.185329346843844
Epoch :  497 / 1000
w1: [14.89063632] w2: [-5.98777252] bias: [16.36533378] loss: 51.18991788224414
Epoch :  498 / 1000
w1

Epoch :  656 / 1000
w1: [16.54392404] w2: [-8.47212691] bias: [16.46564137] loss: 45.66878599304665
Epoch :  657 / 1000
w1: [16.56574021] w2: [-8.49017891] bias: [16.49356957] loss: 45.62581140375064
Epoch :  658 / 1000
w1: [16.57505462] w2: [-8.50630625] bias: [16.50231834] loss: 45.59513088337158
Epoch :  659 / 1000
w1: [16.57467946] w2: [-8.52018179] bias: [16.49489747] loss: 45.57387737139055
Epoch :  660 / 1000
w1: [16.5978131] w2: [-8.52257071] bias: [16.52234086] loss: 45.555807182701564
Epoch :  661 / 1000
w1: [16.6198803] w2: [-8.54250309] bias: [16.52738028] loss: 45.50987035245488
Epoch :  662 / 1000
w1: [16.6213459] w2: [-8.57641235] bias: [16.49834385] loss: 45.45327694167777
Epoch :  663 / 1000
w1: [16.64151556] w2: [-8.58689433] bias: [16.52698224] loss: 45.42627794447933
Epoch :  664 / 1000
w1: [16.64494751] w2: [-8.60396415] bias: [16.52183409] loss: 45.39699083234853
Epoch :  665 / 1000
w1: [16.6661921] w2: [-8.615505] bias: [16.53179666] loss: 45.36557672508216
Epoch

w1: [17.64855943] w2: [-10.82768146] bias: [16.19404946] loss: 41.73863305610618
Epoch :  828 / 1000
w1: [17.66298248] w2: [-10.83905049] bias: [16.20388385] loss: 41.70313679415816
Epoch :  829 / 1000
w1: [17.69478004] w2: [-10.84278035] bias: [16.23895488] loss: 41.63968641888263
Epoch :  830 / 1000
w1: [17.72690071] w2: [-10.8423209] bias: [16.28046108] loss: 41.58210296740684
Epoch :  831 / 1000
w1: [17.75184708] w2: [-10.83636507] bias: [16.31498536] loss: 41.54735554667953
Epoch :  832 / 1000
w1: [17.75831511] w2: [-10.84976554] bias: [16.31334982] loss: 41.52692656358649
Epoch :  833 / 1000
w1: [17.76134205] w2: [-10.86288437] bias: [16.30569425] loss: 41.51289880180854
Epoch :  834 / 1000
w1: [17.78775667] w2: [-10.87290342] bias: [16.32899565] loss: 41.46665249096552
Epoch :  835 / 1000
w1: [17.80305734] w2: [-10.88214293] bias: [16.33941038] loss: 41.43815216341108
Epoch :  836 / 1000
w1: [17.80934873] w2: [-10.88692609] bias: [16.35147114] loss: 41.4227769513677
Epoch :  837

w1: [19.14263507] w2: [-12.71491816] bias: [16.66094505] loss: 38.478979679096
Epoch :  991 / 1000
w1: [19.15544178] w2: [-12.72913248] bias: [16.66882314] loss: 38.462048212323914
Epoch :  992 / 1000
w1: [19.13296553] w2: [-12.75490281] bias: [16.61886403] loss: 38.412228021895416
Epoch :  993 / 1000
w1: [19.15315234] w2: [-12.76295725] bias: [16.63588496] loss: 38.40338609963477
Epoch :  994 / 1000
w1: [19.14828482] w2: [-12.77636257] bias: [16.62010158] loss: 38.38144325070396
Epoch :  995 / 1000
w1: [19.16005194] w2: [-12.78200107] bias: [16.62545587] loss: 38.37289897743575
Epoch :  996 / 1000
w1: [19.14511081] w2: [-12.79693477] bias: [16.59218543] loss: 38.346549913623804
Epoch :  997 / 1000
w1: [19.15421508] w2: [-12.79213851] bias: [16.60595931] loss: 38.354181973038685
Epoch :  998 / 1000
w1: [19.14507558] w2: [-12.80623115] bias: [16.5828923] loss: 38.33223160733884
Epoch :  999 / 1000
w1: [19.15850424] w2: [-12.80799286] bias: [16.61092842] loss: 38.3359040274779
Epoch :  1

### mini-batch 경사 하강법
- iteration마다 전체 데이터를 batch size만큼 쪼개서 weight를 update하는 것을 반복 수행

- a 리스트의 크기가 100인데 a[101]을 하게되면 오류가 뜬다. 
 - 하지만 a[5:120] 이런식으로 슬라이싱을 하게 되면 불러올 수 있는 곳까지 불러오고 오류가 뜨지 않는다

In [6]:
def mini_batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:,0]
    lstat = features[:,1]
    
    for i in range(iter_epochs):
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_step이 0, 30, 60 ... 이런식으로 진행된다
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
            
            bias_update, w1_update, w2_update = get_update_weights_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
            
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
            
            if verbose:
                print('Epoch:',i+1,'/', iter_epochs, 'batch step:', batch_step)
                pred = w1*rm + w2*lstat + bias
                diff = target - pred
                mse_loss = np.mean(np.square(diff))
                print('w1:',w1,'w2:',w2,'bias:',bias,'loss:',mse_loss)
    return w1, w2, bias

In [7]:
w1, w2, bias = mini_batch_gradient_descent(scaled_features, df['price'].values, iter_epochs=1000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #####')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000 batch step: 0
w1: [0.21118173] w2: [0.11625443] bias: [0.41353333] loss: 567.2250927629614
Epoch: 1 / 1000 batch step: 30
w1: [0.42504999] w2: [0.21151116] bias: [0.83382878] loss: 542.7895112848258
Epoch: 1 / 1000 batch step: 60
w1: [0.65529552] w2: [0.29207843] bias: [1.26864819] loss: 518.1392189598428
Epoch: 1 / 1000 batch step: 90
w1: [0.89614794] w2: [0.39426111] bias: [1.69338585] loss: 494.12115632033255
Epoch: 1 / 1000 batch step: 120
w1: [1.02658639] w2: [0.5371618] bias: [1.98799585] loss: 478.0108246502634
Epoch: 1 / 1000 batch step: 150
w1: [1.3122922] w2: [0.6192097] bias: [2.47174746] loss: 452.07615796064226
Epoch: 1 / 1000 batch step: 180
w1: [1.70976219] w2: [0.69994868] bias: [3.0759416] loss: 420.237402226281
Epoch: 1 / 1000 batch step: 210
w1: [2.04599772] w2: [0.79636541] bias: [3.5818841] loss: 394.35643754954594
Epoch: 1 / 1000 batch step: 240
w1: [2.39049798] w2: [0.87851357] bias: [4.08502278] loss: 369.530587101

Epoch: 11 / 1000 batch step: 0
w1: [10.94521657] w2: [0.44230419] bias: [16.12886546] loss: 69.11916982171265
Epoch: 11 / 1000 batch step: 30
w1: [10.95166374] w2: [0.42485375] bias: [16.12801456] loss: 69.07268663652552
Epoch: 11 / 1000 batch step: 60
w1: [10.96421335] w2: [0.42589945] bias: [16.14659203] loss: 69.03045434570946
Epoch: 11 / 1000 batch step: 90
w1: [10.98146772] w2: [0.42187774] bias: [16.16163664] loss: 68.97402608411383
Epoch: 11 / 1000 batch step: 120
w1: [10.94666121] w2: [0.37533716] bias: [16.07898765] loss: 69.04248926648025
Epoch: 11 / 1000 batch step: 150
w1: [11.0226307] w2: [0.37711227] bias: [16.17617378] loss: 68.79169312162352
Epoch: 11 / 1000 batch step: 180
w1: [11.16959712] w2: [0.39828125] bias: [16.38561219] loss: 68.4443068674963
Epoch: 11 / 1000 batch step: 210
w1: [11.26659988] w2: [0.41166702] bias: [16.51065715] loss: 68.30805284870925
Epoch: 11 / 1000 batch step: 240
w1: [11.37308409] w2: [0.42645722] bias: [16.64125632] loss: 68.2310479194291


w1: [13.00645639] w2: [-2.77312194] bias: [16.46175061] loss: 59.21193017182117
Epoch: 19 / 1000 batch step: 450
w1: [12.960642] w2: [-2.81295599] bias: [16.37381363] loss: 59.22899879845466
Epoch: 19 / 1000 batch step: 480
w1: [12.94194925] w2: [-2.8329974] bias: [16.32897185] loss: 59.24314926600614
Epoch: 20 / 1000 batch step: 0
w1: [12.93621395] w2: [-2.85031451] bias: [16.30637882] loss: 59.23434022672556
Epoch: 20 / 1000 batch step: 30
w1: [12.93920962] w2: [-2.86630494] bias: [16.30043355] loss: 59.2047106210061
Epoch: 20 / 1000 batch step: 60
w1: [12.94538289] w2: [-2.86700356] bias: [16.30755326] loss: 59.189290107712516
Epoch: 20 / 1000 batch step: 90
w1: [12.95770434] w2: [-2.8718497] bias: [16.31532225] loss: 59.15688668693804
Epoch: 20 / 1000 batch step: 120
w1: [12.92766583] w2: [-2.90938291] bias: [16.2458808] loss: 59.186845899924926
Epoch: 20 / 1000 batch step: 150
w1: [12.99640154] w2: [-2.90831927] bias: [16.33279897] loss: 59.02201156907341
Epoch: 20 / 1000 batch st

w1: [14.77517623] w2: [-5.75595142] bias: [16.54144081] loss: 51.66541487766638
Epoch: 29 / 1000 batch step: 210
w1: [14.85555136] w2: [-5.74133686] bias: [16.64745674] loss: 51.62930717901327
Epoch: 29 / 1000 batch step: 240
w1: [14.94352721] w2: [-5.72924028] bias: [16.75321863] loss: 51.62960350204172
Epoch: 29 / 1000 batch step: 270
w1: [15.01015519] w2: [-5.72326345] bias: [16.84532491] loss: 51.66572772181688
Epoch: 29 / 1000 batch step: 300
w1: [15.01142073] w2: [-5.72696189] bias: [16.83887883] loss: 51.652292127091734
Epoch: 29 / 1000 batch step: 330
w1: [14.99498603] w2: [-5.73641197] bias: [16.80122998] loss: 51.613927317313696
Epoch: 29 / 1000 batch step: 360
w1: [14.98935647] w2: [-5.77758852] bias: [16.80614017] loss: 51.540547810646544
Epoch: 29 / 1000 batch step: 390
w1: [14.91825581] w2: [-5.86997868] bias: [16.65171284] loss: 51.343517559981386
Epoch: 29 / 1000 batch step: 420
w1: [14.83396364] w2: [-5.95417278] bias: [16.48541584] loss: 51.27333802191052
Epoch: 29 / 

w1: [16.08087393] w2: [-8.25131386] bias: [16.38260507] loss: 46.45441650579276
Epoch: 38 / 1000 batch step: 180
w1: [16.1984029] w2: [-8.23196267] bias: [16.55196919] loss: 46.321661712913986
Epoch: 38 / 1000 batch step: 210
w1: [16.27295031] w2: [-8.21654909] bias: [16.65170873] loss: 46.29850543986523
Epoch: 38 / 1000 batch step: 240
w1: [16.35433635] w2: [-8.20529132] bias: [16.74882454] loss: 46.30426401800059
Epoch: 38 / 1000 batch step: 270
w1: [16.41411671] w2: [-8.20015517] bias: [16.83067436] loss: 46.337381172573
Epoch: 38 / 1000 batch step: 300
w1: [16.41246909] w2: [-8.20422789] bias: [16.81966874] loss: 46.32258807579485
Epoch: 38 / 1000 batch step: 330
w1: [16.3932755] w2: [-8.21414197] bias: [16.77728093] loss: 46.285260132499516
Epoch: 38 / 1000 batch step: 360
w1: [16.39001722] w2: [-8.24564322] bias: [16.79371368] loss: 46.244292158953755
Epoch: 38 / 1000 batch step: 390
w1: [16.32519279] w2: [-8.32693941] bias: [16.65582929] loss: 46.08775042359087
Epoch: 38 / 1000 

...

Epoch: 958 / 1000 batch step: 300
w1: [26.34768613] w2: [-23.35357419] bias: [15.97612655] loss: 30.536991417622275
Epoch: 958 / 1000 batch step: 330
w1: [26.31166924] w2: [-23.36616767] bias: [15.90513773] loss: 30.517070858920874
Epoch: 958 / 1000 batch step: 360
w1: [26.3229427] w2: [-23.336448] bias: [15.99511541] loss: 30.540816375037824
Epoch: 958 / 1000 batch step: 390
w1: [26.29718335] w2: [-23.34816962] bias: [15.96103221] loss: 30.52666901468526
Epoch: 958 / 1000 batch step: 420
w1: [26.24775472] w2: [-23.39158269] bias: [15.86604318] loss: 30.51419667336826
Epoch: 958 / 1000 batch step: 450
w1: [26.2223228] w2: [-23.4092268] bias: [15.82283576] loss: 30.5205904888669
Epoch: 958 / 1000 batch step: 480
w1: [26.20539334] w2: [-23.41068793] bias: [15.79684889] loss: 30.527708948367763
Epoch: 959 / 1000 batch step: 0
w1: [26.19913696] w2: [-23.41158663] bias: [15.7838619] loss: 30.53188611804216
Epoch: 959 / 1000 batch step: 30
w1: [26.19246133] w2: [-23.41160768] bias: [15.77060

Epoch: 967 / 1000 batch step: 0
w1: [26.2022415] w2: [-23.40988092] bias: [15.78170934] loss: 30.531864732001903
Epoch: 967 / 1000 batch step: 30
w1: [26.19556748] w2: [-23.40990164] bias: [15.7684532] loss: 30.536662262401816
Epoch: 967 / 1000 batch step: 60
w1: [26.17397988] w2: [-23.41694751] bias: [15.72737278] loss: 30.556135145606387
Epoch: 967 / 1000 batch step: 90
w1: [26.16740891] w2: [-23.42089714] bias: [15.71208364] loss: 30.564717978116036
Epoch: 967 / 1000 batch step: 120
w1: [26.17724479] w2: [-23.39006801] bias: [15.74863591] loss: 30.54448349279505
Epoch: 967 / 1000 batch step: 150
w1: [26.21176713] w2: [-23.38862492] bias: [15.79274673] loss: 30.526380537003305
Epoch: 967 / 1000 batch step: 180
w1: [26.27926313] w2: [-23.37157233] bias: [15.89512778] loss: 30.514650390604793
Epoch: 967 / 1000 batch step: 210
w1: [26.31586484] w2: [-23.35119629] bias: [15.95399708] loss: 30.526925323406765
Epoch: 967 / 1000 batch step: 240
w1: [26.3545148] w2: [-23.34533493] bias: [15.

Epoch: 977 / 1000 batch step: 90
w1: [26.17121493] w2: [-23.41880646] bias: [15.709457] loss: 30.564687857937702
Epoch: 977 / 1000 batch step: 120
w1: [26.18104996] w2: [-23.38797861] bias: [15.7460075] loss: 30.544455094989644
Epoch: 977 / 1000 batch step: 150
w1: [26.21557293] w2: [-23.38653437] bias: [15.79012146] loss: 30.526352150869023
Epoch: 977 / 1000 batch step: 180
w1: [26.28306706] w2: [-23.36948168] bias: [15.8925008] loss: 30.514624631332346
Epoch: 977 / 1000 batch step: 210
w1: [26.31966528] w2: [-23.34910604] bias: [15.95136634] loss: 30.526899627495006
Epoch: 977 / 1000 batch step: 240
w1: [26.35831192] w2: [-23.34324495] bias: [15.9928644] loss: 30.54543487011146
Epoch: 977 / 1000 batch step: 270
w1: [26.37463318] w2: [-23.3433606] bias: [16.00988038] loss: 30.55515534532089
Epoch: 977 / 1000 batch step: 300
w1: [26.35497876] w2: [-23.34955989] bias: [15.97108096] loss: 30.53694504086786
Epoch: 977 / 1000 batch step: 330
w1: [26.31896666] w2: [-23.36215153] bias: [15.9

w1: [26.37760923] w2: [-23.34172223] bias: [16.00781711] loss: 30.555135090830493
Epoch: 985 / 1000 batch step: 300
w1: [26.35795617] w2: [-23.34792083] bias: [15.96902092] loss: 30.536926472534507
Epoch: 985 / 1000 batch step: 330
w1: [26.32194603] w2: [-23.3605117] bias: [15.89804661] loss: 30.517007951489084
Epoch: 985 / 1000 batch step: 360
w1: [26.33321436] w2: [-23.33079391] bias: [15.9880283] loss: 30.540754104610336
Epoch: 985 / 1000 batch step: 390
w1: [26.30744742] w2: [-23.34252382] bias: [15.95393326] loss: 30.526598810783046
Epoch: 985 / 1000 batch step: 420
w1: [26.25800957] w2: [-23.38594689] bias: [15.85892715] loss: 30.51412460061141
Epoch: 985 / 1000 batch step: 450
w1: [26.23257137] w2: [-23.40359552] bias: [15.81570973] loss: 30.520522352956565
Epoch: 985 / 1000 batch step: 480
w1: [26.21564594] w2: [-23.40505373] bias: [15.78973242] loss: 30.527640193362156
Epoch: 986 / 1000 batch step: 0
w1: [26.20939198] w2: [-23.40595195] bias: [15.77675139] loss: 30.53181635393

w1: [26.2612901] w2: [-23.38414383] bias: [15.85665068] loss: 30.51410207662203
Epoch: 994 / 1000 batch step: 450
w1: [26.23584987] w2: [-23.40179391] bias: [15.81343006] loss: 30.52050108869424
Epoch: 994 / 1000 batch step: 480
w1: [26.21892575] w2: [-23.40325119] bias: [15.78745581] loss: 30.5276187309148
Epoch: 995 / 1000 batch step: 0
w1: [26.21267255] w2: [-23.40414925] bias: [15.77447669] loss: 30.53179456857176
Epoch: 995 / 1000 batch step: 30
w1: [26.20600394] w2: [-23.40416888] bias: [15.76123188] loss: 30.53658849496796
Epoch: 995 / 1000 batch step: 60
w1: [26.18442271] w2: [-23.41121207] bias: [15.72016521] loss: 30.556054262953655
Epoch: 995 / 1000 batch step: 90
w1: [26.17785107] w2: [-23.41516089] bias: [15.70487712] loss: 30.564636168894218
Epoch: 995 / 1000 batch step: 120
w1: [26.18768464] w2: [-23.38433529] bias: [15.74142454] loss: 30.544406409770822
Epoch: 995 / 1000 batch step: 150
w1: [26.2222087] w2: [-23.38288903] bias: [15.78554397] loss: 30.526303485258747
Epo

In [8]:
pred = w1*scaled_features[:,0] + w2*scaled_features[:,1] + bias
df['mini_batch_predicted_price'] = pred
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price,mini_batch_predicted_price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.830077
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.370027
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.508278
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.272631
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.477882


### mini-batch 경사 하강법 keras
- keras는 기본적으로 mini-batch 경사 하강법을 수행한다

In [10]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(1, input_shape=(2,), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, df['price'].values, batch_size=30, epochs=1000)

Epoch 1/1000
17/17 [==============================] - 0s 688us/step - loss: 525.4948 - mse: 525.4948
Epoch 2/1000
17/17 [==============================] - 0s 626us/step - loss: 507.2883 - mse: 507.2883
Epoch 3/1000
17/17 [==============================] - 0s 563us/step - loss: 552.8864 - mse: 552.8864
Epoch 4/1000
17/17 [==============================] - 0s 626us/step - loss: 498.0796 - mse: 498.0796
Epoch 5/1000
17/17 [==============================] - 0s 688us/step - loss: 515.8861 - mse: 515.8861
Epoch 6/1000
17/17 [==============================] - 0s 626us/step - loss: 505.9558 - mse: 505.9558
Epoch 7/1000
17/17 [==============================] - 0s 563us/step - loss: 459.1508 - mse: 459.1508
Epoch 8/1000
17/17 [==============================] - 0s 688us/step - loss: 460.0903 - mse: 460.0903
Epoch 9/1000
17/17 [==============================] - 0s 688us/step - loss: 441.4753 - mse: 441.4753
Epoch 10/1000
17/17 [==============================] - 0s 688us/step - loss: 464.9755 - mse

17/17 [==============================] - 0s 626us/step - loss: 94.4891 - mse: 94.4891
Epoch 82/1000
17/17 [==============================] - 0s 626us/step - loss: 109.2067 - mse: 109.2067
Epoch 83/1000
17/17 [==============================] - 0s 563us/step - loss: 102.7347 - mse: 102.7347
Epoch 84/1000
17/17 [==============================] - 0s 563us/step - loss: 105.6917 - mse: 105.6917
Epoch 85/1000
17/17 [==============================] - 0s 563us/step - loss: 105.3882 - mse: 105.3882
Epoch 86/1000
17/17 [==============================] - 0s 563us/step - loss: 112.8944 - mse: 112.8944
Epoch 87/1000
17/17 [==============================] - 0s 563us/step - loss: 103.0989 - mse: 103.0989
Epoch 88/1000
17/17 [==============================] - 0s 500us/step - loss: 99.1212 - mse: 99.1212
Epoch 89/1000
17/17 [==============================] - 0s 626us/step - loss: 107.0779 - mse: 107.0779
Epoch 90/1000
17/17 [==============================] - 0s 626us/step - loss: 83.5372 - mse: 83.5372


17/17 [==============================] - 0s 500us/step - loss: 64.5787 - mse: 64.5787
Epoch 162/1000
17/17 [==============================] - 0s 563us/step - loss: 59.2322 - mse: 59.2322
Epoch 163/1000
17/17 [==============================] - 0s 500us/step - loss: 62.6983 - mse: 62.6983
Epoch 164/1000
17/17 [==============================] - 0s 563us/step - loss: 63.5315 - mse: 63.5315
Epoch 165/1000
17/17 [==============================] - 0s 563us/step - loss: 51.9359 - mse: 51.9359
Epoch 166/1000
17/17 [==============================] - 0s 500us/step - loss: 68.6003 - mse: 68.6003
Epoch 167/1000
17/17 [==============================] - 0s 563us/step - loss: 72.2976 - mse: 72.2976
Epoch 168/1000
17/17 [==============================] - 0s 626us/step - loss: 62.4008 - mse: 62.4008
Epoch 169/1000
17/17 [==============================] - 0s 626us/step - loss: 56.4982 - mse: 56.4982
Epoch 170/1000
17/17 [==============================] - 0s 500us/step - loss: 58.7817 - mse: 58.7817
Epoch

17/17 [==============================] - 0s 813us/step - loss: 44.0153 - mse: 44.0153
Epoch 243/1000
17/17 [==============================] - 0s 813us/step - loss: 41.6728 - mse: 41.6728
Epoch 244/1000
17/17 [==============================] - 0s 626us/step - loss: 38.8889 - mse: 38.8889
Epoch 245/1000
17/17 [==============================] - 0s 688us/step - loss: 51.9617 - mse: 51.9617
Epoch 246/1000
17/17 [==============================] - 0s 688us/step - loss: 37.9226 - mse: 37.9226
Epoch 247/1000
17/17 [==============================] - 0s 688us/step - loss: 43.7090 - mse: 43.7090
Epoch 248/1000
17/17 [==============================] - 0s 751us/step - loss: 41.7414 - mse: 41.7414
Epoch 249/1000
17/17 [==============================] - 0s 813us/step - loss: 46.4198 - mse: 46.4198
Epoch 250/1000
17/17 [==============================] - 0s 751us/step - loss: 45.2856 - mse: 45.2856
Epoch 251/1000
17/17 [==============================] - 0s 688us/step - loss: 42.7124 - mse: 42.7124
Epoch

17/17 [==============================] - 0s 626us/step - loss: 30.9297 - mse: 30.9297
Epoch 323/1000
17/17 [==============================] - 0s 626us/step - loss: 29.7981 - mse: 29.7981
Epoch 324/1000
17/17 [==============================] - 0s 751us/step - loss: 37.4751 - mse: 37.4751
Epoch 325/1000
17/17 [==============================] - 0s 813us/step - loss: 35.6052 - mse: 35.6052
Epoch 326/1000
17/17 [==============================] - 0s 563us/step - loss: 32.6191 - mse: 32.6191
Epoch 327/1000
17/17 [==============================] - 0s 501us/step - loss: 31.4642 - mse: 31.4642
Epoch 328/1000
17/17 [==============================] - 0s 563us/step - loss: 33.6514 - mse: 33.6514
Epoch 329/1000
17/17 [==============================] - 0s 626us/step - loss: 30.4747 - mse: 30.4747
Epoch 330/1000
17/17 [==============================] - 0s 563us/step - loss: 35.4015 - mse: 35.4015
Epoch 331/1000
17/17 [==============================] - 0s 563us/step - loss: 32.7609 - mse: 32.7609
Epoch

...

17/17 [==============================] - 0s 563us/step - loss: 32.3518 - mse: 32.3518
Epoch 645/1000
17/17 [==============================] - 0s 563us/step - loss: 32.2999 - mse: 32.2999
Epoch 646/1000
17/17 [==============================] - 0s 563us/step - loss: 32.3416 - mse: 32.3416
Epoch 647/1000
17/17 [==============================] - 0s 563us/step - loss: 28.0912 - mse: 28.0912
Epoch 648/1000
17/17 [==============================] - 0s 563us/step - loss: 31.1528 - mse: 31.1528
Epoch 649/1000
17/17 [==============================] - 0s 626us/step - loss: 29.5251 - mse: 29.5251
Epoch 650/1000
17/17 [==============================] - 0s 563us/step - loss: 26.9003 - mse: 26.9003
Epoch 651/1000
17/17 [==============================] - 0s 500us/step - loss: 26.1551 - mse: 26.1551
Epoch 652/1000
17/17 [==============================] - 0s 563us/step - loss: 33.9250 - mse: 33.9250
Epoch 653/1000
17/17 [==============================] - 0s 563us/step - loss: 37.2791 - mse: 37.2791
Epoch

17/17 [==============================] - 0s 563us/step - loss: 30.8537 - mse: 30.8537
Epoch 726/1000
17/17 [==============================] - 0s 563us/step - loss: 35.5163 - mse: 35.5163
Epoch 727/1000
17/17 [==============================] - 0s 500us/step - loss: 25.8285 - mse: 25.8285
Epoch 728/1000
17/17 [==============================] - 0s 626us/step - loss: 29.0612 - mse: 29.0612
Epoch 729/1000
17/17 [==============================] - 0s 563us/step - loss: 31.1250 - mse: 31.1250
Epoch 730/1000
17/17 [==============================] - 0s 563us/step - loss: 32.7587 - mse: 32.7587
Epoch 731/1000
17/17 [==============================] - 0s 563us/step - loss: 33.7140 - mse: 33.7140
Epoch 732/1000
17/17 [==============================] - 0s 500us/step - loss: 29.2018 - mse: 29.2018
Epoch 733/1000
17/17 [==============================] - 0s 563us/step - loss: 28.7353 - mse: 28.7353
Epoch 734/1000
17/17 [==============================] - 0s 563us/step - loss: 26.5107 - mse: 26.5107
Epoch

17/17 [==============================] - 0s 501us/step - loss: 31.8674 - mse: 31.8674
Epoch 807/1000
17/17 [==============================] - 0s 500us/step - loss: 26.6086 - mse: 26.6086
Epoch 808/1000
17/17 [==============================] - 0s 626us/step - loss: 29.4501 - mse: 29.4501
Epoch 809/1000
17/17 [==============================] - 0s 500us/step - loss: 25.8610 - mse: 25.8610
Epoch 810/1000
17/17 [==============================] - 0s 563us/step - loss: 32.1566 - mse: 32.1566
Epoch 811/1000
17/17 [==============================] - 0s 501us/step - loss: 30.0996 - mse: 30.0996
Epoch 812/1000
17/17 [==============================] - 0s 500us/step - loss: 32.5715 - mse: 32.5715
Epoch 813/1000
17/17 [==============================] - 0s 564us/step - loss: 27.6226 - mse: 27.6226
Epoch 814/1000
17/17 [==============================] - 0s 500us/step - loss: 32.2935 - mse: 32.2935
Epoch 815/1000
17/17 [==============================] - 0s 563us/step - loss: 29.7512 - mse: 29.7512
Epoch

17/17 [==============================] - 0s 563us/step - loss: 33.8501 - mse: 33.8501
Epoch 888/1000
17/17 [==============================] - 0s 563us/step - loss: 28.4176 - mse: 28.4176
Epoch 889/1000
17/17 [==============================] - 0s 563us/step - loss: 31.0772 - mse: 31.0772
Epoch 890/1000
17/17 [==============================] - 0s 563us/step - loss: 31.1839 - mse: 31.1839
Epoch 891/1000
17/17 [==============================] - 0s 501us/step - loss: 31.6537 - mse: 31.6537
Epoch 892/1000
17/17 [==============================] - 0s 563us/step - loss: 29.5907 - mse: 29.5907
Epoch 893/1000
17/17 [==============================] - 0s 500us/step - loss: 30.5046 - mse: 30.5046
Epoch 894/1000
17/17 [==============================] - 0s 438us/step - loss: 27.9289 - mse: 27.9289
Epoch 895/1000
17/17 [==============================] - 0s 500us/step - loss: 31.5454 - mse: 31.5454
Epoch 896/1000
17/17 [==============================] - 0s 563us/step - loss: 33.4749 - mse: 33.4749
Epoch

17/17 [==============================] - 0s 563us/step - loss: 28.4644 - mse: 28.4644
Epoch 968/1000
17/17 [==============================] - 0s 500us/step - loss: 29.7215 - mse: 29.7215
Epoch 969/1000
17/17 [==============================] - 0s 563us/step - loss: 29.9534 - mse: 29.9534
Epoch 970/1000
17/17 [==============================] - 0s 563us/step - loss: 29.5707 - mse: 29.5707
Epoch 971/1000
17/17 [==============================] - 0s 500us/step - loss: 29.3328 - mse: 29.3328
Epoch 972/1000
17/17 [==============================] - 0s 563us/step - loss: 29.2633 - mse: 29.2633
Epoch 973/1000
17/17 [==============================] - 0s 563us/step - loss: 30.3464 - mse: 30.3464
Epoch 974/1000
17/17 [==============================] - 0s 563us/step - loss: 26.3914 - mse: 26.3914
Epoch 975/1000
17/17 [==============================] - 0s 501us/step - loss: 31.3470 - mse: 31.3470
Epoch 976/1000
17/17 [==============================] - 0s 501us/step - loss: 31.9827 - mse: 31.9827
Epoch

In [15]:
pred = model.predict(scaled_features)
df['keras_mini_batch_predicted_price'] = pred
df.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price,mini_batch_predicted_price,keras_mini_batch_predicted_price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.830077,28.968491
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.370027,25.493736
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.508278,32.629261
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.272631,32.405685
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.477882,31.593903
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,27.953052,28.095587
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.190621,21.313087
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.655029,17.738962
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,7.975749,8.026713
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.134764,18.234760
